In [10]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

moviesPath = "./ml-latest-small/movies.csv"
ratingsPath = "./ml-latest-small/ratings.csv"
moviesDF = pd.read_csv(moviesPath, index_col=None)
ratingsDF = pd.read_csv(ratingsPath, index_col=None)

trainRatingsDF, testRatingsDF = train_test_split(ratingsDF, test_size=0.2)
print("total_movie_count: " + str(len(set(ratingsDF['movieId'].values.tolist()))))
print("total_user_count: " + str(len(set(ratingsDF['userId'].values.tolist()))))
print("train_movie_count: " + str(len(set(trainRatingsDF['movieId'].values.tolist()))))
print("train_user_count: " + str(len(set(trainRatingsDF['userId'].values.tolist()))))
print("test_movie_count: " + str(len(set(testRatingsDF['movieId'].values.tolist()))))
print("test_user_count: " + str(len(set(testRatingsDF['userId'].values.tolist()))))

total_movie_count: 9724
total_user_count: 610
train_movie_count: 8965
train_user_count: 610
test_movie_count: 5158
test_user_count: 610


In [11]:
trainRatingsPivotDF = pd.pivot_table(testRatingsDF[['userId', 'movieId', 'rating']], columns=['movieId'],
                                     index=['userId'], values='rating', fill_value=0)

In [12]:
moviesMap = dict(enumerate(list(trainRatingsPivotDF.columns)))

usersMap = dict(enumerate(list(trainRatingsPivotDF.index)))

ratingValues = trainRatingsPivotDF.values.tolist()


In [13]:
def calCosinesimilarity(list1, list2):
    res = 0
    denominator1 = 0
    denominator2 = 0
    for (val1, val2) in zip(list1, list2):
        res += (val1 * val2)
        denominator1 += val1 ** 2
        denominator2 += val2 ** 2

    return res / (math.sqrt(denominator1 * denominator2))

In [14]:
# 根据用户对电影的评分来判断每个用户间的相识度
userSimMatrix = np.zeros((len(ratingValues), len(ratingValues)), dtype=np.float32)
for i in range(len(ratingValues) - 1):
    for j in range(i + 1, len(ratingValues)):
        userSimMatrix[i, j] = calCosinesimilarity(ratingValues[i], ratingValues[j])
        userSimMatrix[j, i] = userSimMatrix[i, j]

In [15]:
# 找到与每个用户最相近的前K个用户
userMostSimDict = dict()
for i in range(len(ratingValues)):
    userMostSimDict[i] = sorted(enumerate(list(userSimMatrix[i])), key=lambda x: x[1], reverse=True)[:10]


In [16]:
# 用这K个用户的喜好中目标用户没有看过的电影进行推荐
userRecommendValues = np.zeros((len(ratingValues), len(ratingValues)), dtype=np.float32)

for i in range(len(ratingValues)):
    for j in range(len(ratingValues[i])):
        if ratingValues[i][j] == 0:
            val = 0
            for (user, sim) in userMostSimDict[i]:
                val += (ratingValues[user][j] * sim)
            userRecommendValues[i, j] = val

IndexError: index 610 is out of bounds for axis 1 with size 610

In [ ]:
userRecommendDict = dict()
for i in range(len(ratingValues)):
    userRecommendDict[i] = sorted(enumerate(list(userRecommendValues[i])), key=lambda x: x[1], reverse=True)[:10]


In [ ]:
# 将一开始的索引转换为原来用户id与电影id
userRecommendList = []
for key, value in userRecommendDict.items():
    user = usersMap[key]
    for (movieId, val) in value:
        userRecommendList.append([user, moviesMap[movieId]])


In [17]:
# 将推荐结果的电影id转换成对应的电影名
recommendDF = pd.DataFrame(userRecommendList, columns=['userId', 'movieId'])
recommendDF = pd.merge(recommendDF, moviesDF[['movieId', 'title']], on='movieId', how='inner')
print(recommendDF.tail(10))

NameError: name 'userRecommendList' is not defined